# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import os
from dotenv import load_dotenv
import sys

In [2]:
load_dotenv()

lib_dir = os.environ.get("LIB_DIR")
sys.path.append(lib_dir)

data_dir = os.environ.get("DATA_DIR")
model_dir = os.path.join(data_dir, "models")

points_filename = "points_final.csv"
measurements_filename = "measurements.csv"

In [3]:
import pandas as pd
import torch
import yaml
from pinn import PINN
import utils

In [4]:
device = utils.get_device()
print(f"Device: {device}")

Device: mps


In [5]:
wing_df = pd.read_csv(os.path.join(data_dir, points_filename))
measurements_df = pd.read_csv(os.path.join(data_dir, measurements_filename))

In [6]:
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [7]:
# Air Density (rho): 1.2041kg/m^3
# Air Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = config["rho"]
mu = config["mu"]

# m/s
in_velocity = config["in_velocity"]

# Domain limits
x_max = config["x_max"]
y_max = config["y_max"]
z_max = config["z_max"]

In [8]:
input_dim = 3
output_dim = 4
hidden_units = config["hidden_units"]

model_name = config["model_name"]

pinn = PINN(input_dim, output_dim, hidden_units, model_name).to(device)

In [9]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1, line_search_fn="strong_wolfe")
# optimizer = torch.optim.Adam(pinn.parameters(), lr=1e-4)

In [10]:
epochs = config["epochs"]
checkpoint_epochs = config["checkpoint_epochs"]

Nf = config["Nf"]   # num of collocation points -> pde evaluation
Nb = config["Nb"]   # num of points to evaluate boundary conditions
Nw = config["Nw"]   # num of points of the surface of the front wing to evaluate boundary conditions
Nu = config["Nu"]   # num of points of real data

In [11]:
pinn.train_pinn(
    epochs=epochs, 
    optimizer=optimizer, 
    s_df=wing_df, 
    u_df=measurements_df, 
    Nf=Nf, 
    Nb=Nb, 
    Ns=Nw, 
    Nu=Nu, 
    x_max=x_max, 
    y_max=y_max, 
    z_max=z_max, 
    in_velocity=in_velocity, 
    mu=mu, 
    rho=rho, 
    device=device, 
    checkpoint_epochs=checkpoint_epochs, 
    model_dir=model_dir,
    c2=100, c3=100, c4=100, c5=100, c6=100, c8=100, c9=100,
)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=1000, bias=True)
    (1-2): 2 x Linear(in_features=1000, out_features=1000, bias=True)
    (3): Linear(in_features=1000, out_features=4, bias=True)
  )
)
Model name: v30
Number of epochs: 3000
Number of collocation points Nf: 100
Number of boundary condition points Nb: 100
Number of object surface points Ns: 100
Number of real data points Nu: 2
X max: 1, Y max: 1, Z max: 1
mu: 1, rho: 1
c1: 1.0, c2: 100, c3: 100, c4: 100, c5: 100, c6: 100, c7: 1.0, c8: 100, c9: 100, c10: 1.0
Inflow velocity: 10
Device: mps
Checkpoint epochs: 50
Model directory: /Users/ggito/repos/pinns/data/front_wing/models
=> converting Nf, Nb, Ns, Nu to nearest power of 2...
Nf: 128, Nb: 128, Ns: 128, Nu: 2
=> starting training...
Epoch: 1, Total Loss: 1321.2983, PDE Loss - Navier Stoker: 104.0296, PDE Loss - Poisson: 0.3990, BC Inlet Loss: 7.1191, BC Outlet Loss: 0.0001, BC Left Loss: 0.1173, BC Right Loss: 0.1332, BC Down Loss: 3.0425, BC Up

In [ ]:
# pinn, optimizer = pinn.load_checkpoint_num(optimizer, model_dir, model_name, 2000)